In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./samples.tsv", sep="\t")
df

In [ ]:

# df = df.drop(["Epitope Embedding", "TRA_CDR3 Embedding", "TRB_CDR3 Embedding"], axis=1)
# df = df.drop(["Unnamed: 0"], axis=1)
# df


In [ ]:
# df.to_csv("./samples.tsv", sep="\t", index=False)

In [ ]:
import numpy as np

epitope_embeddings_path = "./paired/paired_epitope_embeddings.npz"
tra_embeddings_path = "./paired/paired_TRA_CDR3_embeddings.npz"
trb_embeddings_path = "./paired/paired_TRB_CDR3_embeddings.npz"

# Load the entire DataFrame
data_frame = df

# Memory-map the embeddings
epitope_embeddings = np.load(epitope_embeddings_path)
tra_embeddings = np.load(tra_embeddings_path)
trb_embeddings = np.load(trb_embeddings_path)

data_frame["Epitope Embedding"] = data_frame["Epitope"].map(epitope_embeddings)
data_frame["TRA_CDR3 Embedding"] = data_frame["TRA_CDR3"].map(tra_embeddings)
data_frame["TRB_CDR3 Embedding"] = data_frame["TRB_CDR3"].map(trb_embeddings)

columns = list(data_frame.columns)  # Get the list of all column names
columns.remove('Binding')
columns.append('Binding')
data_frame = data_frame[columns]

In [ ]:
data_frame

In [ ]:
df = df[["Epitope Embedding", "TRA_CDR3 Embedding", "TRB_CDR3 Embedding"]]
df

In [ ]:
epitope_embeddings = []
tra_cdr3_embeddings = []
trb_cdr3_embeddings = []

for _, row in df.iterrows():
    epitope_embeddings.append(row["Epitope Embedding"])
    tra_cdr3_embeddings.append(row["TRA_CDR3 Embedding"])
    trb_cdr3_embeddings.append(row["TRB_CDR3 Embedding"])

max_length = max(
    max(embedding.shape[0] for embedding in epitope_embeddings),
    max(embedding.shape[0] for embedding in tra_cdr3_embeddings),
    max(embedding.shape[0] for embedding in trb_cdr3_embeddings)
)

max_length

In [ ]:
# Correct iteration over a Series
for index, value in df['Epitope Embedding'].items():
    print(f"Index: {index}, Value: {value.shape}")

In [ ]:
# Correct iteration over a Series
for index, value in df['TRA_CDR3 Embedding'].items():
    print(f"Index: {index}, Value: {value.shape}")

In [ ]:
# Correct iteration over a Series
for index, value in df['TRB_CDR3 Embedding'].items():
    print(f"Index: {index}, Value: {value.shape}")

In [ ]:
import torch

# function used for the padding of the embeddings
def pad_collate(batch):
    # Initialize lists to hold the embeddings and labels
    epitope_embeddings, tra_cdr3_embeddings, trb_cdr3_embeddings, labels = [], [], [], []

    # Extract embeddings and labels from the batch
    for item in batch:
        epitope_embeddings.append(item["epitope_embedding"])
        tra_cdr3_embeddings.append(item["tra_cdr3_embedding"])
        trb_cdr3_embeddings.append(item["trb_cdr3_embedding"])
        labels.append(item["label"])


    # TODO instead of max the batch we need to max to the entire dataset 
    # otherwise we find a way to write this max_length into this train.py. 
    # However if i overwrite IN_CHANNELS in this function here does not work because gets executed in Dataloader...
    # Find the maximum length across all embeddings in the batch
    max_length = max(
        max(embedding.size(0) for embedding in epitope_embeddings),
        max(embedding.size(0) for embedding in tra_cdr3_embeddings),
        max(embedding.size(0) for embedding in trb_cdr3_embeddings)
    )

    # Function to pad embeddings to the maximum length
    def pad_embeddings(embeddings):
        return torch.stack([
            torch.nn.functional.pad(embedding, (0, 0, 0, max_length - embedding.size(0)), "constant", 0)
            for embedding in embeddings
        ])

    # Pad and stack all embeddings and labels
    epitope_embeddings = pad_embeddings(epitope_embeddings)
    tra_cdr3_embeddings = pad_embeddings(tra_cdr3_embeddings)
    trb_cdr3_embeddings = pad_embeddings(trb_cdr3_embeddings)
    labels = torch.stack(labels)

    print(f"this is in pad_collate: epitope_embedding:\n{epitope_embeddings.shape}")
    return {
        "epitope_embedding": epitope_embeddings,
        "tra_cdr3_embedding": tra_cdr3_embeddings,
        "trb_cdr3_embedding": trb_cdr3_embeddings,
        "label": labels
    } 

In [ ]:
torch.tensor(df["Epitope Embedding"][0], dtype=torch.float32).detach()

In [ ]:
list_epitope_embeddings = []

list_epitope_embeddings.append(torch.tensor(df["Epitope Embedding"][0], dtype=torch.float32))

list_epitope_embeddings

In [ ]:
list_epitope_embeddings[0].shape[0]

In [ ]:
max_length = 20
def pad_embeddings(embeddings):
        return torch.stack([
            torch.nn.functional.pad(embedding, (0, 0, 0, max_length - embedding.shape[0]), "constant", 0)
            for embedding in embeddings
        ])

stack = pad_embeddings(list_epitope_embeddings)

In [ ]:
stack.shape

In [ ]:
print(stack)